# Explainability of Random Forest Model

Install the follwoing packages:
- numpy
- pandas
- scikit-learn
- lime

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
import time
from sklearn.metrics import classification_report
from lime.lime_tabular import LimeTabularExplainer
from sklearn.inspection import PartialDependenceDisplay
import matplotlib.pyplot as plt
import shap

## Setting up and training RandomForestClassification model

In [ ]:
session = get_active_session()

session.use_database("ML")
session.use_schema("RETAIL_STORE")

df_training_data = session.table('training_data') # importing data

df_training_data = df_training_data.drop("CUSTOMER_ID", "OFFER_PRODUCT_ID") # dropping id columns
X = df_training_data.drop("REPEATER_INT")
y = df_training_data.select("REPEATER_INT")


FEATURE_COLS = X.columns[:len(X.columns)]
LABEL_COLS = ["REPEATER_INT"]

print(f"Feature Columns: {FEATURE_COLS}")

X = X.to_pandas()
y = y.to_pandas()

y = y.values.ravel()

# 80/20 train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Feature selection testing

In [ ]:
# Drop CHAIN_CAT_1, 2, and 3
# Drop OFFER_VALUE 2, 3, 4
X.drop(columns=['CHAIN_CAT_1', 'CHAIN_CAT_2', 'CHAIN_CAT_3', 'OFFER_VALUE_2', 'OFFER_VALUE_3', 'OFFER_VALUE_4'])
print(X.head())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Making TOTAL binary

If total > 5800, then 1 otherwise 0. 
Result:
- Gives it a feature importance of 0.0

In [ ]:
# Making the total binary, above 5853 or not
X['TOTAL'] = X['TOTAL'].apply(lambda x: 1 if x > 5800 else 0)

print(X['TOTAL'].head())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Training model

In [ ]:
start_time = time.time()

model = RandomForestClassifier(
    n_estimators = 50,
    min_samples_split = 5,
    min_samples_leaf = 1,
    max_features = 'log2',
    max_depth = 10, 
    class_weight = 'balanced'
)

model.fit(X_train, y_train)

end_time = time.time()
training_time = end_time - start_time

### Parameter optimisation example


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

model = RandomForestClassifier()


param_dist = {
    'n_estimators': [25, 50, 100, 200],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt', 'log2'], # dont use 'auto'. error
}


random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=5, ###s
    scoring='recall',
    cv=5,
    verbose=2,
    random_state=42,
)

random_search.fit(X_train, y_train)

end_time = time.time()
training_time = end_time - start_time

print("Training time: ", training_time) 

parameters = random_search.best_params_
print(parameters)

## Feature importance

In [ ]:
model = RandomForestClassifier(
    n_estimators = 50,
    min_samples_split = 5,
    min_samples_leaf = 1,
    max_features = 'log2',
    max_depth = 10, # Changed
    class_weight = 'balanced'
)

model.fit(X_train, y_train)

In [ ]:

importances = model.feature_importances_

FEATURE_COLS = X.columns.tolist()

features = dict(zip(FEATURE_COLS, importances))
                
importances = dict(sorted(features.items(), key=lambda item: item[1], reverse=True))

for i, im in enumerate(importances.items()):
    print(f"{i+1}. Feature: {im[0]} - importance: {im[1]}")

## LIME

Local Interpretable Model-agnostic Explanations


How it works

It "zooms" in on a given date point and then creates new datapoints around it, a process called perturbation. Then it takes an easily explainable model such as a linear model and trains it on the new data points around. This then is used to inform how much each feature impacts the ouptut at various values. So for this model where there are no continous values, you can see to what degree each feature impacts the prediction depending on whether they are 0 or 1. 

In [ ]:
explainer = LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=X_train.columns,
    #categorical_features=X_train.columns,
    mode='classification'
)

In [ ]:
import warnings
from random import randint

# Ignore all warnings
warnings.filterwarnings('ignore')

# Choose an instance to explain
instance_index = randint(0, 1500)
instance = X_test.iloc[instance_index]

explanation = explainer.explain_instance(
    data_row=instance,
    predict_fn=model.predict_proba
)

print(f"Instance index: {instance_index}")
explanation.as_pyplot_figure()

# 2000 < > 5853

## PDP

What does it measure? For each feature it measures the average prediction if all data points were to assume that feature value. So for this model, it wil take a feature and assume it is 0 and then calculate the average probability of 1 across all data points, and then do the same for 1. A flat PDP indicates that the feature is not important, and the more it varies, the more improtant the feature is. Meaning that if there is a large difference in the prediction between the two values one of the feature takes, that feature has a large impact. 

The plots below are linear because there is only two possible values on the x-axis, 0 and 1. If I was using continous variables in the model it would be a plotted non-linear(perhaps) line.  

Seems more usable for continous variables as you can then see for what values there is a bigger change in the output. If you had age for example, there might be a marked change in your risk for cardiac disease when you hit 60. Easy to see with a PD plot. 

Slow to run with continous variables!

In [ ]:
print(FEATURE_COLS)

In [ ]:
FEATURE_COLS = X.columns.tolist()
display = PartialDependenceDisplay.from_estimator(model, X_train, FEATURE_COLS)

display.plot(pdp_lim={1: (0, 1)})

plt.show()

In [ ]:


for i, feature in enumerate(FEATURE_COLS):
     plt.figure(figsize=(3, 3))
     display = PartialDependenceDisplay.from_estimator(model, X_train, [feature])
     display.plot(pdp_lim={0: (0, 1)})
     plt.title(f'Partial Dependence Plot for {feature}')
     plt.xlabel(feature)
     plt.ylabel('Partial Dependence')
     plt.grid(True)
     plt.show()


## Storing model results


In [ ]:
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix

start_time = time.time()

predictions = model.predict(X_test)

end_time = time.time()
prediction_time = end_time - start_time

parameters = model.get_params()

accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy:.4f}')

recall = recall_score(y_test, predictions)
print(f'Recall: {recall:.4f}')

conf_matrix = confusion_matrix(y_test, predictions)
print('Confusion Matrix:')
print(conf_matrix)

# For storing in db
true_positive = conf_matrix[0][0]  
true_negative = conf_matrix[1][1]  
false_positive = conf_matrix[0][1]  
false_negative = conf_matrix[1][0]

In [ ]:
import json

def record_performance(true_positive, true_negative, false_positive, false_negative, model_name, accuracy, recall, training_time, prediction_time, 
         parameters, coefficients, intercept, notes):

    confusion_matrix_insert_sql = f"""
        insert into model_results_schema.confusion_matrix
        (true_positive, true_negative, false_positive, false_negative)
        values
        ({true_positive}, {true_negative}, {false_positive}, {false_negative});
    """
    
    session.sql(confusion_matrix_insert_sql).collect()

    last_id_sql = """
        select id
        from model_results_schema.confusion_matrix
        order by create_at desc
        limit 1;
    """ 

    #SELECT LAST_VALUE(id) OVER (ORDER BY id RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS last_id
    confusion_matrix_id = session.sql(last_id_sql).collect()
    confusion_matrix_id = confusion_matrix_id[0]['ID']

    # These two need to be on a string format.
    if coefficients != "":
        coefficients = ', '.join(map(str, coefficients))
    parameters = json.dumps(parameters)
    
    # Insert data into the model_performance table
    session.sql(f"""
        insert into model_results_schema.model_performance
            (model_name, accuracy, recall, confusion_matrix_id,
            training_time, prediction_time, parameters, coefficients,
            intercept, notes)
        values
            ('{model_name}', {accuracy}, {recall}, {confusion_matrix_id}, {training_time}, {prediction_time}, '{parameters}', '{coefficients}', {intercept}, '{notes}');
    """).collect()

    
    return "success"



In [ ]:
notes = "depth=10 instead of 30"
model_name = "RandomForestClassifier+total-depth10"

result = record_performance(true_positive, true_negative, false_positive, false_negative, model_name, accuracy, recall, training_time, prediction_time, 
         parameters, [0], 0.0, notes)
print(result)

## SHAP

Calculating Shapley Values

In [ ]:
# Small dataset
X_small = X
y_small = y
X_train, X_test, y_train, y_test = train_test_split(X_small, y_small, test_size=0.2, random_state=42)

In [ ]:
start_time = time.time()

model = RandomForestClassifier(
    n_estimators = 50,
    min_samples_split = 5,
    min_samples_leaf = 1,
    max_features = 'log2',
    max_depth = 10, 
    class_weight = 'balanced'
)

model.fit(X_train, y_train)

end_time = time.time()
training_time = end_time - start_time

In [ ]:
explainer = shap.Explainer(model.predict, X_test)

shap_values = explainer(X_test)
shap_values

In [ ]:
shap.plots.bar(shap_values)

In [ ]:
shap.plots.beeswarm(shap_values)

Bar plot
- mean shap value

In [ ]:
shap.plots.bar(shap_values[23])

### Note on SHAP in snowflake
- Unable to display force plots
- Unable to display dependence_plots, but scatter plots work.

In [ ]:
shap.plots.scatter(shap_values[:,"OFFER_VALUE_6"], color=shap_values[:,"OFFER_VALUE_6"])